# **Chains**

chains are a fundamental concept that allows you to execute complex tasks in a structured and efficient way.

## **Why Chains?**

Chains are invaluable due to their capacity to effortlessly blend diverse components, shaping a singular and coherent application. Through the creation of chains, multiple elements can seamlessly come together. Imagine this scenario: a chain is crafted to take in user input, polish it using a PromptTemplate, and subsequently pass on this refined response to a large language model (LLM). This streamlined process not only simplifies but also enriches the overall functionality of the system. In essence, chains serve as the linchpin, seamlessly connecting different parts of the application and enhancing its capabilities.


In [1]:
%%capture
# update or install the necessary libraries
%pip install --upgrade langchain_classic langchain_community langchain_aws python-dotenv 

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = os.getenv("AWS_DEFAULT_REGION")

# **LLM Chain** - **The simplest chain**

The LLMChain is a foundational system that includes a PromptTemplate, an OpenAI model (such as a Large Language Model or a ChatModel), and optionally, an output parser. It operates by transforming input parameters using the PromptTemplate into a coherent prompt, which is then fed into the model. The resulting output is further refined and formatted into a usable form by the OutputParser, if provided. This structured approach ensures effective utilization of language models for various applications, enhancing their functionality and utility.

In [3]:
from langchain_classic.prompts import ChatPromptTemplate
from langchain_classic.chains import LLMChain

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.5
)

In [5]:
prompt = ChatPromptTemplate.from_template(
    "What are the tools that need to be learned to earn the {designation}?,provide me one tool"
)

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/rm/gvl01dcd5dg9dxwkn8zgg9v40000gn/T/ipykernel_16774/1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [7]:
designation = "Devops Engineer"
chain.invoke(designation)

{'designation': 'Devops Engineer',
 'text': " To become a DevOps Engineer, you need to have a good understanding of various tools and technologies that enable continuous integration, continuous delivery, infrastructure automation, and monitoring. Here are some of the most commonly used tools in the DevOps field:\n\n1. Version Control Systems (VCS) - Git is the most popular version control system used in DevOps.\n2. Continuous Integration (CI) Tools - Jenkins, Travis CI, CircleCI, and GitLab CI are popular CI tools.\n3. Containerization Tools - Docker and Kubernetes are widely used containerization tools.\n4. Configuration Management Tools - Ansible, Puppet, and Chef are popular configuration management tools.\n5. Infrastructure as Code (IaC) Tools - Terraform, CloudFormation, and Azure Resource Manager are popular IaC tools.\n6. Monitoring and Logging Tools - Nagios, Zabbix, Prometheus, and ELK Stack are popular monitoring and logging tools.\n7. Orchestration Tools - Apache Airflow, Lu

# **SimpleSequentialChain**

Simple Sequential Chains allow for a single input to undergo a series of coherent transformations, resulting in a refined output. This sequential approach ensures systematic and efficient handling of data, making it ideal for scenarios where a linear flow of information processing is essential

In [8]:
# SimpleSequentialChain
from langchain_classic.chains import SimpleSequentialChain
from langchain_classic.prompts import ChatPromptTemplate
from langchain_classic.chains import LLMChain

In [9]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What are the tools that need to be learned to earn the {designation}?,provide me one tool"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    tool:{tool_name1}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:

# LC -> chain_one -> LLM -> output tool_name

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=False
                                            )

In [15]:
designation = "Devops Engineer"
overall_simple_chain.invoke(designation)

{'input': 'Devops Engineer',
 'output': ' Learn Git for version control, the foundation of DevOps. Understand distributed development, branching, merging, and tracking changes. Explore other tools like Docker for containerization, configuration management tools, CI/CD systems, IaC tools, and monitoring/logging systems for continuous integration, delivery, and deployment.'}

# **SequentialChain**

A sequential chain is a chain that combines various individual chains, where the output of one chain serves as the input for the next in a continuous sequence. It operates by running a series of chains consecutively.

In [16]:
# SequentialChain
from langchain_classic.chains import SequentialChain

In [17]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following designation"
    "\n\n{Designation}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="Designation_description"
                    )


In [18]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following description in 1 sentence:"
    "\n\n{Designation_description}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [19]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What are the promgramming language that need to be learned for this designation in one line:\n\n{Designation}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="programming_language"
                      )

In [20]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a short follow up response to the following "
    "Write a short summary about the programming language:"
    "\n\nSummary: {summary}\n\nLanguage: {programming_language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [ ]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
#Designation = "Devops Engineer"
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Designation"],
    output_variables=["Designation_description","programming_language", "summary","followup_message"],
    verbose=False
)

In [23]:
designation = "Deveops Engineer"
overall_chain.invoke(designation)

{'Designation': 'Deveops Engineer',
 'Designation_description': ' A DevOps Engineer is a software professional responsible for designing, developing, and implementing continuous delivery pipelines, automating infrastructure, and ensuring high system performance and availability.',
 'programming_language': ' To excel as a DevOps Engineer, proficiency in infrastructure automation tools like Terraform, Docker, Kubernetes, and configuration management tools such as Ansible, Puppet, or Chef, coupled with strong scripting skills in Python, Bash, or Ruby, is essential. Familiarity with version control systems (Git), CI/CD tools (Jenkins, Travis CI), and container orchestration platforms is also important.',
 'summary': ' A DevOps Engineer is a software expert who designs, develops, and implements continuous delivery pipelines, automates infrastructure, and guarantees high system performance and availability.',
 'followup_message': " Response: I'd like to add that the DevOps Engineer's scripti

# **Let's Do an Activity**

## **Objective**

Practice using chains with language models and structured output parsing to handle complex tasks efficiently.

## **Scenario**

You are tasked with building a system to assist users in understanding various technical concepts related to software engineering roles. Your goal is to use different chains to process user queries, transform them using prompt templates, and extract specific information using output parsers.

## **Steps**

* Define a Prompt Template
* Prepare Sample Queries
* Implement LLM Chains
* Output Parsing
* Interactive Chain Execution
* Evaluate